In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
from tvDatafeed import TvDatafeed, Interval
from IPython.display import display, HTML

# Baixar dados de fechamento da PETR4
petr4_ohlc = yf.download('PETR4.SA', start='2020-01-01', end='2024-05-31')
petr4_dividendos = yf.Ticker('PETR4.SA').dividends

# Ajustar índice para datetime
petr4_dividendos.index = pd.to_datetime(petr4_dividendos.index.date)

# Baixar dados do IBOV do Yahoo Finance
ibov_yf = yf.download('^BVSP', period='max')
ibov_yf = ibov_yf.rename(columns={'Close': 'close_yf'})

# Exemplo 1: Join simples
df_simple_merge = pd.merge(petr4_ohlc['Close'], petr4_dividendos, how='right', left_index=True, right_index=True)

# Baixar dados do IBOV do Trading View
tv = TvDatafeed()
ibov_tv = tv.get_hist(symbol='IBOV', exchange='BMFBOVESPA', interval=Interval.in_daily, n_bars=10000)
ibov_tv.index = pd.to_datetime(ibov_tv.index.date)
ibov_tv.index.name = 'Date'
ibov_tv = ibov_tv.rename(columns={'close': 'close_tv'})

start_date = '2020-01-01'
end_date = '2024-05-31'
ibov_tv = ibov_tv[start_date:end_date]

# Exemplo 2: Join com indicator=True
df_advanced_merge = pd.merge(ibov_yf['close_yf'], ibov_tv['close_tv'], how='outer', left_index=True, right_index=True, indicator=True)

# Exemplo 3: Join com 'on' e 'suffixes'
ibov_yf['Data'] = ibov_yf.index
ibov_tv['Data'] = ibov_tv.index
df_merge_on_suffixes = pd.merge(ibov_yf[['Data', 'close_yf']], ibov_tv[['Data', 'close_tv']], how='inner', on='Data', suffixes=['_yf', '_tv'])

# Exemplo 4: Join com 'validate'
df_merge_validate = pd.merge(ibov_yf[['Data', 'close_yf']], ibov_tv[['Data', 'close_tv']], how='inner', on='Data', suffixes=['_yf', '_tv'], validate='one_to_one')

# HTML para exibir os DataFrames
html = f"""
<style>
    .dataframe-container {{
        display: flex;
        flex-wrap: wrap;
        justify-content: space-around;
    }}
    .dataframe-container div {{
        flex: 1;
        margin: 10px;
    }}
    h3 {{
        text-align: left;
        color: yellow;
    }}
</style>
<div class="dataframe-container">
    <div>
        <h3>Join simples de dados: PETR4 'fechamento' mesclado com 'dividendos'</h3>
        {df_simple_merge.head().to_html()}
    </div>
    <div>
        <h3>Join com 'indicator': Mesclados os dados IBOV de duas fontes diferentes</h3>
        {df_advanced_merge.head().to_html()}
    </div>
</div>
<div class="dataframe-container">
    <div>
        <h3>Join com 'on' e 'suffixes'</h3>
        {df_merge_on_suffixes.head().to_html()}
    </div>
    <div>
        <h3>Join com 'validate' - Verificar se a junção está ocorrendo corretamente</h3>
        {df_merge_validate.head().to_html()}
    </div>
</div>
"""

display(HTML(html))


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
